<a href="https://colab.research.google.com/github/Maruf-16203091/Flowers-Prediction/blob/main/flower_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [23]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [24]:
from google.colab import drive
drive.mount('/content/drive')

dataset = ImageFolder(root='/content/drive/MyDrive/ML_Datasets/flowers', transform=transform)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)

In [26]:
classes = dataset.classes
print(f"Classes: {classes}")

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=len(classes)):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)     # 64x64x3 → 64x64x16
        self.pool = nn.MaxPool2d(2, 2)                 # 64x64 → 32x32
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)    # 32x32x16 → 32x32x32
        self.fc1 = nn.Linear(32 * 16 * 16, 128)         # Flatten and FC
        self.fc2 = nn.Linear(128, num_classes)          # Output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Classes: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [27]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")


Epoch 1/20, Loss: 0.4424, Accuracy: 84.00%
Epoch 2/20, Loss: 0.4294, Accuracy: 83.54%
Epoch 3/20, Loss: 0.4158, Accuracy: 84.75%
Epoch 4/20, Loss: 0.3807, Accuracy: 85.68%
Epoch 5/20, Loss: 0.3350, Accuracy: 88.28%
Epoch 6/20, Loss: 0.3830, Accuracy: 86.34%
Epoch 7/20, Loss: 0.3409, Accuracy: 88.14%
Epoch 8/20, Loss: 0.3136, Accuracy: 88.25%
Epoch 9/20, Loss: 0.3286, Accuracy: 88.17%
Epoch 10/20, Loss: 0.2735, Accuracy: 90.16%
Epoch 11/20, Loss: 0.2789, Accuracy: 90.13%
Epoch 12/20, Loss: 0.2670, Accuracy: 89.93%
Epoch 13/20, Loss: 0.2600, Accuracy: 90.31%
Epoch 14/20, Loss: 0.2380, Accuracy: 91.17%
Epoch 15/20, Loss: 0.2174, Accuracy: 92.42%
Epoch 16/20, Loss: 0.2229, Accuracy: 91.87%
Epoch 17/20, Loss: 0.2196, Accuracy: 92.42%
Epoch 18/20, Loss: 0.2148, Accuracy: 92.65%
Epoch 19/20, Loss: 0.2112, Accuracy: 92.82%
Epoch 20/20, Loss: 0.1889, Accuracy: 93.46%


In [34]:
model.eval()
val_correct = 0
val_total = 0
with torch.no_grad():
    for val_inputs, val_labels in test_loader:
        val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
        val_outputs = model(val_inputs)
        _, val_predicted = torch.max(val_outputs, 1)
        val_total += val_labels.size(0)
        val_correct += (val_predicted == val_labels).sum().item()

val_accuracy = 100 * val_correct / val_total
print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, "
      f"Train Accuracy: {100*correct/total:.2f}%, Test Accuracy: {val_accuracy:.2f}%")

Epoch 20/20, Loss: 0.1889, Train Accuracy: 93.46%, Test Accuracy: 71.45%


In [ ]:
from google.colab import files
from PIL import Image

# Upload image from local
uploaded = files.upload()

# Get the uploaded filename
image_path = list(uploaded.keys())[0]

# Load and preprocess image
image = Image.open(image_path).convert("RGB")
image = transform(image).unsqueeze(0).to(device)

# Run the model
model.eval()
with torch.no_grad():
    output = model(image)
    _, predicted = torch.max(output, 1)

# Get predicted class name
class_name = dataset.classes[predicted.item()]
print(f"Predicted class: {class_name}")